# Tutorial 81: Groups

This example demonstrates how add, remove, set elements to groups.

# SIR 3S Installation

In [38]:
SIR3S_SIRGRAF_DIR = r"C:\3S\SIR 3S\SirGraf-90-15-00-21_Quebec-Upd2" #change to local path

# Imports

In [39]:
from sir3stoolkit.core import wrapper

In [40]:
from sir3stoolkit.mantle import mantle

The wrapper package has to be initialized with reference to a SIR 3S (SirGraf) installation.

In [41]:
wrapper.Initialize_Toolkit(SIR3S_SIRGRAF_DIR)

## Additional imports

In [42]:
import os
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import contextily as cx

...

# Initialization

In [43]:
s3s = mantle.SIR3S_Model_Mantle()

Initialization complete


# Open Model

In [44]:
dbFilePath=r"Toolkit_Tutorial81_Model.db3"

In [45]:
s3s.OpenModel(dbName=dbFilePath,
              providerType=s3s.ProviderTypes.SQLite,
              Mid="M-1-0-1",
              saveCurrentlyOpenModel=False,
              namedInstance="",
              userID="",
              password="")

Model is open for further operation


# View current state of groups

In [46]:
group_tks = [] # tks of group objects

In [47]:
tks_of_group_elements = [] # tks of elements inside groups

We can obtain the tks part of individual groups with the function [get_tks_of_group_elements()](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.mantle.html#sir3stoolkit.mantle.advanced_operations.SIR3S_Model_Advanced_Operations.get_tks_of_group_elements).

In [48]:
for index, tk in enumerate(s3s.GetTksofElementType(s3s.ObjectTypes.LAYR_Layer)):
    tks = s3s.get_tks_of_group_elements(tk)
    print(f"{index}: {tk}: {s3s.GetValue(tk, 'Name')[0]}")
    tks_of_group_elements.append(tks)
    group_tks.append(tk)

0: 5566604953098438347: Vorlauf
1: 4645068345115859295: Rücklauf
2: 5499257419847606859: Rohre


In [49]:
tks_of_group_elements[0]

[('KNOT', '5428054456958551597'),
 ('KNOT', '5099111544186125239'),
 ('ROHR', '4742112892005639406')]

In [50]:
tks_of_group_elements[1]

[('KNOT', '5743596851943449715'),
 ('KNOT', '5269793380005646745'),
 ('ROHR', '5531803347555043206'),
 ('KNOT', '5629554381987149893'),
 ('ROHR', '5164228635035739777')]

Note that when using this function we get lists of tuples containing both the element type and tk of each element. The reason for this is that when adding elements to other classes, we need to now both their element type and tk.

# Remove 

The last two elements in the Rücklauf (return) group were mistakenly placed there and actually belong to the Vorlauf (supply) group. First, we remove them from the Rücklauf group.

In [51]:
tks_to_be_moved = [tks_of_group_elements[1][i] for i in [3,4]]

In [52]:
tks_to_be_moved

[('KNOT', '5629554381987149893'), ('ROHR', '5164228635035739777')]

Now we have the desired format of tks and can use it with the function [remove_elements_from_group()](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.mantle.html#sir3stoolkit.mantle.advanced_operations.SIR3S_Model_Advanced_Operations.remove_elements_from_group). 

In [53]:
s3s.remove_elements_from_group(group_tk=group_tks[1], element_tks=tks_to_be_moved,)

[2026-01-25 19:08:39,353] INFO in sir3stoolkit.mantle.advanced_operations: [remove elements from group] Check successful


Value is set


We can check whether the elements were removed

In [54]:
s3s.get_tks_of_group_elements(group_tk=group_tks[1])

[('KNOT', '5743596851943449715'),
 ('KNOT', '5269793380005646745'),
 ('ROHR', '5531803347555043206')]

# Add tk list to group (supply)

We can now add them to the Vorlauf (supply) group using [add_elements_to_group()](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.mantle.html#sir3stoolkit.mantle.advanced_operations.SIR3S_Model_Advanced_Operations.remove_elements_from_group).

In [55]:
s3s.add_elements_to_group(group_tk=group_tks[0], element_tks=tks_to_be_moved)

[2026-01-25 19:08:39,435] INFO in sir3stoolkit.mantle.advanced_operations: [add elements to group] Check successful


Value is set


We can check whether they were added.

In [56]:
s3s.get_tks_of_group_elements(group_tk=group_tks[0])

[('KNOT', '5428054456958551597'),
 ('KNOT', '5099111544186125239'),
 ('ROHR', '4742112892005639406'),
 ('KNOT', '5629554381987149893'),
 ('ROHR', '5164228635035739777')]

Furthermore we have two tks given, that are outside any group, but we want them to be part of the Vorlauf (supply) group. We dont have their element types saved with them. We can use the [add_element_types_to_tk_list()](###) function to add the element types. Note that these element types are not directley equivalent to the s3s.ObjectTypes. This function only works for District Heating network, because for others the element types would carry different names.

In [57]:
tks_given = ["4651757704563987050", "5520138268724292357"]

In [58]:
tks_given_with_element_types = s3s.add_element_types_to_tk_list(tks_given)

In [59]:
tks_given_with_element_types

[('ROHR', '4651757704563987050'), ('KNOT', '5520138268724292357')]

Now we have the desired format of tks and can use it with the function [add_elements_to_group()](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.mantle.html#sir3stoolkit.mantle.advanced_operations.SIR3S_Model_Advanced_Operations.remove_elements_from_group). 

In [60]:
s3s.add_elements_to_group(group_tk=group_tks[0], element_tks=tks_given_with_element_types)

[2026-01-25 19:08:39,532] INFO in sir3stoolkit.mantle.advanced_operations: [add elements to group] Check successful


Value is set


We can check whether the elements were added.

In [61]:
s3s.get_tks_of_group_elements(group_tk=group_tks[0])

[('KNOT', '5428054456958551597'),
 ('KNOT', '5099111544186125239'),
 ('ROHR', '4742112892005639406'),
 ('KNOT', '5629554381987149893'),
 ('ROHR', '5164228635035739777'),
 ('ROHR', '4651757704563987050'),
 ('KNOT', '5520138268724292357')]

# Overwrite Group tk list

Let's say we want to fill the group Rohre (global list of all pipes). As is is empty at the moment.

In [62]:
s3s.get_tks_of_group_elements(group_tk=group_tks[2])

[]

In [63]:
pipe_tks = []

In [64]:
for group_tk in group_tks:
    element_tks_with_type = s3s.get_tks_of_group_elements(group_tk=group_tk)
    for element in element_tks_with_type:
        if element[0] == 'ROHR':
            pipe_tks.append(element)

In [65]:
pipe_tks = list(set(pipe_tks)) # ensure uniqueness

In [66]:
s3s.set_group_elements(group_tk=group_tks[2], element_tks=pipe_tks)

[2026-01-25 19:08:39,627] INFO in sir3stoolkit.mantle.advanced_operations: [set elements for group] Check successful


Value is set


In [67]:
s3s.get_tks_of_group_elements(group_tk=group_tks[2])

[('ROHR', '4742112892005639406'),
 ('ROHR', '5164228635035739777'),
 ('ROHR', '4651757704563987050'),
 ('ROHR', '5531803347555043206')]

In [68]:
s3s.CloseModel(False)

True

# Possible Issue

If we were to add a node from the Rücklauf (return) group to the Vorlauf (supply) group, but accidentley use 'ROHR' instead of 'KNOT', it would be added to the tks of the group, as there is no safe guard against this.

In [69]:
s3s.OpenModel(dbName=dbFilePath,
              providerType=s3s.ProviderTypes.SQLite,
              Mid="M-1-0-1",
              saveCurrentlyOpenModel=False,
              namedInstance="",
              userID="",
              password="")

Model is open for further operation


In [70]:
s3s.add_elements_to_group(group_tk=group_tks[0], element_tks=[('ROHR', '5269793380005646745')])

[2026-01-25 19:08:43,689] INFO in sir3stoolkit.mantle.advanced_operations: [add elements to group] Check successful


Value is set


In [71]:
s3s.get_tks_of_group_elements(group_tk=group_tks[0])

[('KNOT', '5428054456958551597'),
 ('KNOT', '5099111544186125239'),
 ('ROHR', '4742112892005639406'),
 ('ROHR', '5269793380005646745')]

In [72]:
s3s.SaveChanges()

Changes saved successfully


But after saving the changes, it would we removed automatically from the list.

In [73]:
s3s.get_tks_of_group_elements(group_tk=group_tks[0])

[('KNOT', '5428054456958551597'),
 ('KNOT', '5099111544186125239'),
 ('ROHR', '4742112892005639406')]

In [74]:
s3s.CloseModel(False)

True